# IST664 - Homework 3

Originality assertion: All of the text and comments in this file are my original work (except for template items written by the instructor). All of the code in this file is my work, except where I give credit to another source. By adding my name below, I affirm this originality assertion.

*** My name: _Darrell_Collison_ ***


**Task 1: Read Sentence Pairs from Github**

It is a weird acronym, but the "Sentences Involving Compositional Knowledge" (SICK) dataset includes a large number of sentence pairs with various levels of similarity. Here's a link to a web page with more information about the data:
https://marcobaroni.org/composes/sick.html

When you review the data, you will notice a field where the options are ENTAILMENT, CONTRADICTION, or NEUTRAL. Textual entailment is when one sentence logically follows based on the meaning of another sentence. You can read more about textual entailment here: https://en.wikipedia.org/wiki/Textual_entailment

In [2]:
import pandas as pd

sick_data = pd.read_csv("SICK_train.txt" , sep='\t', on_bad_lines='skip', index_col=None)

# HW3T1A
# Add code to display the first few rows of data

sick_data.head()



,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [ ]:
# HW3T1B
# Add code that shows the number of variables and the number of rows in the dataset.
# Add some comments describing what each of the columns contains.
# The columns are the pair id, 2 sentences A and B, how related each sentence is, and its entailment (where the sentences depend on each other)

sick_data.shape


(4500, 5)

**Task 2: Add a Similarity Score to Each Row**

Use a pre-trained sentence embedding model to generate a similarity score for each sentence pair. To get started, here's some code from Lab 7:

In [6]:
# HW3T2A
# The first thing we will need is the library for loading sentence transformers
# This generates a lot of output, but should run pretty fast.
!pip install sentence-transformers


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: C:\Users\Black Knight\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [7]:
# HW3T2B
# Now load a pre-trained sentence transformer. There are hundreds to choose from.
# This downloads a lot of data to your virtual machine and takes half a minute or so.
from sentence_transformers import SentenceTransformer

# Here's a sentence transformer model that encodes a d=384 vector. See:
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

c:\Users\Black Knight\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Black Knight\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\Black Knight\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violatio

c:\Users\Black Knight\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Black Knight\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' packag

In [8]:
# HW3T2C
# Here's an example of how we can put the sentence transformer to use
# by encoding the two sentences from the first row and computing a
# cosine similarity between them.
from sentence_transformers.util import cos_sim

show_row = 2

print(sick_data["sentence_A"][show_row])
print(sick_data["sentence_B"][show_row])
print(sick_data["entailment_judgment"][show_row])
a = model.encode([sick_data["sentence_A"][show_row]])
b = model.encode([sick_data["sentence_B"][show_row]])

cos_sim(a, b).tolist()[0][0] # The notation on the end extracts a scalar from the tensor object

The young boys are playing outdoors and the man is smiling nearby
The kids are playing outdoors near a man with a smile
ENTAILMENT


0.8005356192588806

In [9]:
# HW3T2D
# Add code to produce a similarity score for each sentence pair and insert the
# list into the Pandas dataframe as a new column. You can use the "insert" method
# on a Pandas df to add a column. Choose a sensible label for your new variable as
# you will need it later for the regression analysis.
#
# Note that if you process the whole dataset, it will take a couple of minutes
# to encode all 4500 of the sentences. You can subset the data down to 1000
# or 1500 rows if you prefer to shorten the run time.

import torch

a = model.encode(sick_data["sentence_A"].tolist())
b = model.encode(sick_data["sentence_B"].tolist())

similarities_tensor = cos_sim(torch.from_numpy(a), torch.from_numpy(b))
similarity_scores = [similarities_tensor[i][i].item() for i in range(len(sick_data))]

sick_data.insert(loc=5, column='Cosine_Similarity_Score', value=similarity_scores)

In [10]:
# HW3T2E
# Review a few rows of the modified Pandas df with the new column
#
sick_data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment,Cosine_Similarity_Score
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL,0.851040
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL,0.540574
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT,0.800536
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL,0.608034
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL,0.489728


In [11]:
# HW3T2F
# Run and interpret a simple correlation between your new similarity score
# and the original relatedness_score column. The corr() method from Pandas
# (which is a method for columns) can do this.

sick_data['relatedness_score'].corr(sick_data['Cosine_Similarity_Score'])

np.float64(0.8415403591981562)

Add your interpretation of the correlation by replacing this text. Would you consider this value to be small? Large? Is it what you expected? Why?

The correlation R2 is 0.84. This is a very strong correlation but was expected. I assumed that related score for the 2 sentences was just another form of similarity. 

**Task 3: Create Dummy Codes for Entailment**

In this task after this one, you will produce a regression analysis that predicts the original relatedness_score from your newly added similarity measure along with two dummy codes created from the entailment_judgment.

With linear regression we can use as predictors any combination of metric variables and binary codes. With a label like entailment_judgment, with three or more categories, we need to create dummy codes to represent the categories. To avoid collinearity, there should always be one less dummy code than the number of categories.

In [14]:
# HW3T3A
# Show the three types of entailment_judgment using set().
set(sick_data['entailment_judgment'])

{'CONTRADICTION', 'ENTAILMENT', 'NEUTRAL'}

In [16]:
# HW3T3B
contra = [ int(ej == "CONTRADICTION") for ej in sick_data["entailment_judgment"]]
entail = [ int(ej == "ENTAILMENT") for ej in sick_data["entailment_judgment"]]

In [17]:
# HW3T3C
# Insert the two new dummy codes into your pandas df

sick_data['is_contradiction'] = contra
sick_data['is_entailment'] = entail

In [21]:
# HW3T3D
# Review the newly modified df

sick_data.head(15)


,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment,Cosine_Similarity_Score,is_contradiction,is_entailment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL,0.851040,0,0
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL,0.540574,0,0
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT,0.800536,0,1
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL,0.608034,0,0
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL,0.489728,0,0
5,12,Two dogs are fighting,Two dogs are wrestling and hugging,4.0,NEUTRAL,0.670884,0,0
6,14,A brown dog is attacking another animal in fro...,Two dogs are fighting,3.5,NEUTRAL,0.564386,0,0
7,18,A brown dog is attacking another animal in fro...,Two dogs are wrestling and hugging,3.2,NEUTRAL,0.411986,0,0
8,25,Nobody is riding the bicycle on one wheel,A person in a black jacket is doing tricks on ...,2.8,NEUTRAL,0.201614,0,0
9,26,A person is riding the bicycle on one wheel,A man in a black jacket is doing tricks on a m...,3.7,NEUTRAL,0.306472,0,0


**Task 4: See if the root verbs match**

In HW2 you conducted some information extraction from sentences including locating the root verb. If the root verbs match on two sentences, that may indicate something about how related the sentences are. For this task you will create a list of dummy codes that indicate whether the two sentences in each pair use the same root verb.

In [22]:
# HW3T4A
import spacy # SpaCy will help us find roots
nlp = spacy.load("en_core_web_sm")

In [23]:
# HW3T4B
# Here's some demo code that shows how to fetch the roots
show_row = 5

print(sick_data["sentence_A"][show_row])
print(sick_data["sentence_B"][show_row])

doc1 = nlp(sick_data["sentence_A"][show_row])
doc2 = nlp(sick_data["sentence_B"][show_row])

list(doc1.sents)[0].root, list(doc2.sents)[0].root

Two dogs are fighting
Two dogs are wrestling and hugging


(fighting, wrestling)

In [24]:
# HW3T4C
# Here's a test that compares the two strings
list(doc1.sents)[0].root.text == list(doc2.sents)[0].root.text

False

In [31]:
# HW3T4D
# Write a loop that will perform the test on each pair of sentences and save the result in a list

nlp = spacy.load("en_core_web_sm")
my_list=[] 

for index, row in sick_data.iterrows():
    doc1 = nlp(row['sentence_A'])
    doc2 = nlp(row['sentence_B'])

    results = list(doc1.sents)[0].root.text == list(doc2.sents)[0].root.text

    my_list.append(results)


In [34]:
print(my_list)
len(my_list)

[True, True, True, True, True, False, False, False, False, False, False, False, False, True, True, True, True, False, False, True, False, False, False, False, True, False, True, False, False, False, False, True, False, False, False, True, False, True, False, True, False, True, True, True, True, True, False, False, False, False, True, True, True, False, True, False, False, False, True, False, False, True, False, False, True, True, False, False, False, True, False, False, True, True, True, False, True, True, True, False, True, False, False, False, False, True, True, False, False, True, True, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, False, False, True, False, False, False, False, False, True, False, False, True, False, False, True, True, False, False, False, True, True, False, True, True, True, False, True, True, True, False, False, False, True, True, True, True, False, False, True, True, False, False, False, True, True, False, False, Fa

4500

In [35]:
# HW3T4E
# Insert the new dummy code into your pandas df.
# Use the column name, "root_match", as shown in the example

sick_data.insert(loc=6, column='root_match', value= my_list)

In [37]:
# HW3T4F
# Display the pandas df

sick_data.head(10)

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment,Cosine_Similarity_Score,root_match,is_contradiction,is_entailment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL,0.851040,True,0,0
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL,0.540574,True,0,0
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT,0.800536,True,0,1
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL,0.608034,True,0,0
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL,0.489728,True,0,0
5,12,Two dogs are fighting,Two dogs are wrestling and hugging,4.0,NEUTRAL,0.670884,False,0,0
6,14,A brown dog is attacking another animal in fro...,Two dogs are fighting,3.5,NEUTRAL,0.564386,False,0,0
7,18,A brown dog is attacking another animal in fro...,Two dogs are wrestling and hugging,3.2,NEUTRAL,0.411986,False,0,0
8,25,Nobody is riding the bicycle on one wheel,A person in a black jacket is doing tricks on ...,2.8,NEUTRAL,0.201614,False,0,0
9,26,A person is riding the bicycle on one wheel,A man in a black jacket is doing tricks on a m...,3.7,NEUTRAL,0.306472,False,0,0


**Task 5: Run Regression Analysis to Predict relatedness_score**

Next, your job is to conduct a regression analysis where you predict relatedness_score from your newly added similarity measure along with two dummy codes created from the entailment_judgment. The sklearn library has a linear regression model, but statsmodels.api produces nicer output.

In [41]:
# HW3T5A
# The sm.OLS() method does regression and once you have fitted the model
# the summary() method produces nice output with all the values needed to
# interpret regression results.
#
import statsmodels.api as sm
import statsmodels.formula.api as smf

# You can specify your regression model using a formula language, like this:
results = smf.ols('relatedness_score ~ Cosine_Similarity_Score + entailment_judgment + is_contradiction + root_match', data=sick_data).fit()
# Of course, you may need to adjust the X-variable names depending upon the
# labels you used when you inserted them into the database.


In [42]:
# HW3T5B
# Use the summary() method on your results object to show the regression output.
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      relatedness_score   R-squared:                       0.775
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     3090.
Date:                Mon, 01 Dec 2025   Prob (F-statistic):               0.00
Time:                        22:18:27   Log-Likelihood:                -3144.1
No. Observations:                4500   AIC:                             6300.
Df Residuals:                    4494   BIC:                             6339.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          9.374e+11   1.41e+12      0.666      0.506   -1.82e+12     3.7e+12
entailment_judgment[T.ENTAILMENT] -9.374e+11   1.41e+12     -0.666      0.506    -3.7e+12    1.82e+12
entailment_judgment[T.NEUTRAL]    -9.374e+11   1.41e+12     -0.666      0.506    -3.7e+12    1.82e+12
root_match[T.True]                   -0.0855      0.017     -5.161      0.000      -0.118      -0.053
Cosine_Similarity_Score               2.8237      0.038     75.296      0.000       2.750       2.897
is_contradiction                  -9.374e+11   1.41e+12     -0.666      0.506    -3.7e+12    1.82e+12
==============================================================================
Omnibus:                       10.834   Durbin-Watson:                   1.602
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               12.759
Skew:                          -0.043   Prob(JB):                      0.00170
Kurtosis:                       3.246   Cond. No.                     5.55e+14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.99e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

HW3T5C

Replace this text with an explanation of the regression output. Make sure to report on the values of the coefficients for the predictors (and the intercept). Also report and interpret the R-squared value. Is this a good predictive model?

This is a relatively good model with an Adj R2 of 0.774. This means that the predictors listed accounts for about 77% of the variation we see in the dependent variable relatedness_score. The intercept is 9.374 x 10 ^ 11, and the coefficients for the 2 significant variables root_match and Cosine_Similarity_Score are -0.085 and 2.824 respectively. 

**Task 6: Find the sentence with the largest error of prediction and the smallest error of prediction.**



In [ ]:
# HW3T6A
# Use the predict() method on the results object to produce predicted scores
# for each row in your data frame. For the regression equation to work, you must
# supply all four of the X variables for each case. Pandas makes it
# easy to create a Pandas subset of columns, like this:
#
# predictors = sick_data[['similarity', 'dummy_entail', 'dummy_contra', 'root_match']]


In [ ]:
# HW3T6B
import matplotlib.pyplot as plt

#
# Show a histogram of predicted values from the previous block. Comment on
# why the range of the predicted values does or does not make sense.
#

In [ ]:
# HW3T6C
# Compute a squared error value for each row of your data frame. This is easy:
# subtract the predicted value from the actual value (i.e., relatedness_score)
# and square the result. You can use a for loop or a list comprehension.

# Hint 1:
# Python's built-in zip() function allows you to bundle two vectors of data
# together into a list of tuples.
# Hint 2:
# When extracting data from a Pandas column, sometimes you must use the
# tolist() method to get the data into a structure that can be used for
# functions like zip().



In [ ]:
# HW3T6D
import numpy as np
# Use np.argmin() to find the index of the row in the Pandas dataset with the
# smallest squared error of prediction


In [ ]:
# HW3T6E
# Use np.argmax() to find the index of the row with the largest squared error


In [ ]:
# HW3T6F
# Display the Pandas data from the row with the worst prediction. Note that
# the iloc() method allows you to select a particular row.


In [ ]:
# HW3T6G
# Display the Pandas data from the row with the best prediction


HW3T6H

Replace this text with an explanation of why the argmin() case was "easy" to predict and the argmax() case was "hard" to predict. This explanation should consider the semantic content of the sentences in reference to the relatedness_score developed by the experts.

**Task 7: Write your own two contradictory sentences and predict the relatedness score**

In [ ]:
# HW3T7A
# Replace these empty strings with your own sentences
test_sentenceA = ""
test_sentenceB = ""

In [ ]:
# HW3T7B
# Encode the sentences into vectors as was demonstrated for task 2.


In [ ]:
# HW3T7C
# Compute the cosine similarity between the two vectors.

In [ ]:
# HW3T7D
# You may have used the formula interface for fitting your regression model
# with smf.ols(). If that's true, the easiest way to get a prediction for a
# novel case is to build a one-row Pandas dataframe and enter the X variables
# you need to make the prediction. The next two lines of commented code
# provide an exmaple of this technique:
# temp_df = pd.DataFrame(columns=['similarity', 'dummy_entail', 'dummy_contra', 'root_match'])
# temp_df.loc[0] = [0.80, 0, 0, 0] # You can include whatever X data you want in place of these three values
#


In [ ]:
# HW3T7E
# Use the predict() method on your regression output object to make one
# prediction of relatedness_score using the one-row Pandas dataframe from
# the previous step.
#


HW3T7F

Replace this text with an interpretation of the output score from the prediction based on your two sentences.

HWCC

Don't forget to write a concluding comment describing your findings. Provide an overview of your findings from the tasks you accomplished above. How well were you able to predict the relatedness score from the other variables? Did entailment or contradiction work as predictors and what does each coefficient say about our ability to predict relatedness scores? Would it be helpful to know something about entailment if we are trying to understand paraphrases? Why do we care about entailment in the context of an application such as chat?